# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


1820
[['Sydney Youngblood', 'Logged In', 'Jacob', 'M', '53', 'Klein', '238.07955', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '954', "Ain't No Sunshine", '200', '1.54345E+12', '73'], ['Gang Starr', 'Logged In', 'Layla', 'F', '88', 'Griffin', '151.92771', 'paid', 'Lake Havasu City-Kingman, AZ', 'PUT', 'NextSong', '1.54106E+12', '984', 'My Advice 2 You (Explicit)', '200', '1.54345E+12', '24'], ['3OH!3', 'Logged In', 'Layla', 'F', '89', 'Griffin', '192.522', 'paid', 'Lake Havasu City-Kingman, AZ', 'PUT', 'NextSong', '1.54106E+12', '984', 'My First Kiss (Feat. Ke$ha) [Album Version]', '200', '1.54345E+12', '24'], ['RÃ\x83Â¶yksopp', 'Logged In', 'Jacob', 'M', '54', 'Klein', '369.81506', 'paid', 'Tampa-St. Petersburg-Clearwater, FL', 'PUT', 'NextSong', '1.54056E+12', '954', 'The Girl and The Robot', '200', '1.54345E+12', '73'], ['Kajagoogoo', 'Logged In', 'Layla', 'F', '90', 'Griffin', '223.55546', 'paid', 'Lake Havasu City-Kingman, AZ', 'PUT', 'NextSong

In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

1505


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## This table addresses the requirement:provide the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
drop_query = "DROP TABLE IF EXISTS session_analytics "
query = "CREATE TABLE IF NOT EXISTS session_analytics "
#Primary key consists of session_id as Partition key and item_in_session as Clustering Column.The data will be distributed through the nodes by 
#session_id (Partition key) and wil be ordered by item_in_session. session_id & item_in_session guarantees the uniqness of every record 
query = query + "(session_id int,item_in_session int,artist_name text,song_name text,song_length float, PRIMARY KEY ((session_id),item_in_session))"
try:
    session.execute(drop_query)
    session.execute(query)
except Exception as e:
    print(e)

                    

In [9]:
# set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into session_analytics(session_id , item_in_session , artist_name ,song_name ,song_length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
query = 'SELECT artist_name ,song_name ,song_length FROM session_analytics where session_Id = 338 and item_In_Session = 4'
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_name, row.song_length)

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [11]:
## This table addresses the requirements:provide name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
drop_query = "DROP TABLE IF EXISTS user_session_analytics "
query = "CREATE TABLE IF NOT EXISTS user_session_analytics "
#Primary key consists of composite partition key (user_id,session_id)  and item_in_session as Clustering Column.The data will be distributed through the nodes by 
#user_id,session_id (Partition key) and wil be ordered by item_in_session so that to provide song (sorted by itemInSession) as required . user_id ,session_id & item_in_session guarantees the uniqness of every record 
query = query + "(user_id int, session_id int, item_in_session int, artist_name text, song_name text ,user_first_name text, user_last_name text  , PRIMARY KEY ((user_id,session_id),item_in_session))"
try:
    session.execute(drop_query)
    session.execute(query)
except Exception as e:
    print(e)

                    

In [12]:
# set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into user_session_analytics(user_id , session_id , item_in_session , artist_name , song_name  ,user_first_name , user_last_name )"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9],line[1],line[4]))

In [13]:
query = "SELECT artist_name ,song_name ,user_first_name , user_last_name   FROM user_session_analytics where user_id = 10 and session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist_name, row.song_name, row.user_first_name,row.user_last_name  )

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


In [14]:
## This table addresses this requirements:Provide every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
drop_query = "DROP TABLE IF EXISTS user_x_song "
query = "CREATE TABLE IF NOT EXISTS user_x_song "
#Primary key consists of simple partition key (song_name)  and user_id as Clustering Column.The data will be distributed through the nodes by 
#song_name (Partition key). user_id is used as clustering column to provide uniqueness for song_name and user_id so that if user listened to a song more than one time
#it will appear as one record as per the requirements.
query = query + "(song_name text,user_id int ,user_first_name text, user_last_name text  , PRIMARY KEY ((song_name),user_id))"
try:
    session.execute(drop_query)
    session.execute(query)
except Exception as e:
    print(e)

                    

In [15]:
# set up the CSV file.
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "insert into user_x_song(song_name, user_id, user_first_name, user_last_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]),line[1],line[4]))

In [18]:
query = "SELECT user_first_name, user_last_name   FROM user_x_song where song_name='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print ( row.user_first_name,row.user_last_name )

### Drop the tables before closing out the sessions

In [19]:
drop_query_1 = "drop table session_analytics"
drop_query_2 = "drop table user_session_analytics"
drop_query_3 = "drop table user_x_song"
try:
    rows = session.execute(drop_query_1)
    rows = session.execute(drop_query_2)
    rows = session.execute(drop_query_3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [20]:
session.shutdown()
cluster.shutdown()